# In-Class Activity: WordNet Schemas

In this assignment, you will be getting hands-on experience with working with WordNet and manipulating the data.

In the problems below, pretend that you (or, rather, your agent) are in the middle of playing a text adventure game. Different characters are asking for various items, but you don't have the exact items. You will be using WordNet to form schemas of your inventory items and going your representations to find the most similar item.

## Stanza

I've setup Stanford's Stanza parser for parsing sentences. Install it and download the models here. You might need to restart the runtime afterward.

In [1]:
%%capture
!python -m pip install stanza
import stanza
stanza.download('en')

### Calling Stanza

In [2]:
import stanza
import json
nlp = stanza.Pipeline('en', processors='pos,lemma,tokenize,ner,depparse, mwt, constituency')

# parsing an example sentence
parse = nlp("How appropriate, you fight like a cow.")
y = json.loads(str(parse))
y

[[{'deprel': 'advmod',
   'end_char': 3,
   'feats': 'PronType=Int',
   'head': 2,
   'id': 1,
   'lemma': 'how',
   'ner': 'O',
   'start_char': 0,
   'text': 'How',
   'upos': 'ADV',
   'xpos': 'WRB'},
  {'deprel': 'advcl',
   'end_char': 15,
   'feats': 'Degree=Pos',
   'head': 5,
   'id': 2,
   'lemma': 'appropriate',
   'ner': 'O',
   'start_char': 4,
   'text': 'appropriate',
   'upos': 'ADJ',
   'xpos': 'JJ'},
  {'deprel': 'punct',
   'end_char': 16,
   'head': 5,
   'id': 3,
   'lemma': ',',
   'ner': 'O',
   'start_char': 15,
   'text': ',',
   'upos': 'PUNCT',
   'xpos': ','},
  {'deprel': 'nsubj',
   'end_char': 20,
   'feats': 'Case=Nom|Number=Sing|Person=2|PronType=Prs',
   'head': 5,
   'id': 4,
   'lemma': 'you',
   'ner': 'O',
   'start_char': 17,
   'text': 'you',
   'upos': 'PRON',
   'xpos': 'PRP'},
  {'deprel': 'root',
   'end_char': 26,
   'feats': 'Mood=Ind|Number=Sing|Person=2|Tense=Pres|VerbForm=Fin',
   'head': 0,
   'id': 5,
   'lemma': 'fight',
   'ner': 'O',

# WordNet


Below is an example of how WordNet functions. Dog is a hyponym (offspring) of canine, which is a hyponym of carnivore. So canine is a hypernym (ancestor) of dog.

![See text above](https://drive.google.com/uc?id=1qhhg-CtXzN8yDcdQCx7ydkEnKYVX4AUj)

Each word has multiple dictionary entries with different meanings. Each one is called a "Synset" and is given a tag. For example, Synset('dog.n.01') where dog is the word, "n" is the part of speech (noun), and 01 is the definition number (this refers to the definition for the animal _dog_).


You can play with it online here: http://wordnetweb.princeton.edu/perl/webwn


### Citation:
```
@article{miller1995wordnet,
  title={WordNet: a lexical database for English},
  author={Miller, George A.},
  journal={Communications of the ACM},
  volume={38},
  number={11},
  pages={39--41},
  year={1995},
  publisher={ACM New York, NY, USA}
}
```

### Using WordNet: NLTK

# Follow [this link](https://www.nltk.org/howto/wordnet.html) to a tutorial on using NLTK for WordNet.

In [3]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

## The Task: Create a schema for your inventory

In the problems below, an NPC is asking for a particular item, but you don't have anything in your inventory with that name.

For each item in your inventory, I'd like you to create a schema---some way of formalizing the information. And you will write code to look through each schema to see which inventory item is appropriate to use in the scenario by comparing the information in the schemas to the request.

Admittedly, this isn't the fastest way of comparing words, since you can just do the following, but I want you to practice making schemas!

In [6]:
from nltk.corpus import wordnet as wn

In [7]:
# The quick way to do it
dog = wn.synsets('dog')
cat = wn.synsets('cat')[0]
print(dog) #see all possible definitions of "dog"
dog = dog[0] #select the first one
print(dog.path_similarity(cat, simulate_root=False)) #find similarity between the two words

[Synset('dog.n.01'), Synset('frump.n.01'), Synset('dog.n.03'), Synset('cad.n.01'), Synset('frank.n.02'), Synset('pawl.n.01'), Synset('andiron.n.01'), Synset('chase.v.01')]
0.2


Fill out the code below. Keep the WordNet online demo open to help you figure out what you should be saving.

### Problem 1

In [76]:
# your inventory
inventory = ["tuna", "glasses", "salmon", "fish", "sushi", "shoe"]

#TODO create a representation of each of these items
inv_schema = {}
for i in inventory:
  inv_item = wn.synsets(i) #TODO: look up the synset for the word
  inv_facts = []
  for ss in inv_item:
      inv_facts += ss.hypernyms() 
      inv_facts += ss.hyponyms()
  #TODO: fill inv_facts with information from Wordnet relevant to your schema
  # Store hypernyms, hyponymns, part meronym, or whatever you need to find the answer
  # You might have to go a couple words deep
  inv_schema[i] = inv_facts

# Added by yifei
smallest_similarity_score = 1000
closest_inventory_item = ''
closest_inventory_item_schema = ''

npc_request = nlp("Do you have any sockeye?")
request_json = json.loads(str(npc_request))[0]
for word in request_json:
  print(word['lemma'])
  #TODO: compare to your schemas
  # Yifei: Should catch the key word more automatically, but here for convenience
  if word['lemma'] == 'sockeye':
    target_item_synset_list =  wn.synsets(word['lemma']) 
    for inventory_item, inventory_item_synset_list in inv_schema.items():
      for target_item_synset in target_item_synset_list:
        for inventory_item_synset in inventory_item_synset_list:
          similarity_score = target_item_synset.path_similarity(inventory_item_synset)
        #   print(target_item_synset, inventory_item_synset, similarity_score)  
          if similarity_score is not None:
            if similarity_score < smallest_similarity_score:
                smallest_similarity_score = similarity_score
                closest_inventory_item = inventory_item            
                closest_inventory_item_schema = [target_item_synset, inventory_item_synset]

#TODO: print the closest inventory item and what part of the schema was used to compare it 
print('Clostest inventory item:', closest_inventory_item)
print('Clostest inventory item shema:', closest_inventory_item_schema[0], 'and', closest_inventory_item_schema[1])

do
you
have
any
sockeye
?
Clostest inventory item: tuna
Clostest inventory item shema: Synset('red_salmon.n.01') and Synset('scombroid.n.01')


### Problem 2

In [84]:
from nltk.corpus import wordnet as wn

# your inventory
inventory = ["oranges", "lipstick", "tree", "garden", "sap", "diagram"]

#TODO create a representation of each of these items
inv_schema = {}
for i in inventory:
  inv_item = wn.synsets(i) #TODO: look up the synset for the word
  inv_facts = []
  for ss in inv_item:
      inv_facts += ss.hypernyms() 
      inv_facts += ss.hyponyms()
  #TODO: fill inv_facts with information from Wordnet relevant to your schema
  # Store hypernyms, hyponymns, part meronym, or whatever you need to find the answer
  # You might have to go a couple words deep
  inv_schema[i] = inv_facts

# Added by yifei
smallest_similarity_score = 1000
closest_inventory_item = ''
closest_inventory_item_schema = []

npc_request = nlp("Do you have any sticks?")
request_json = json.loads(str(npc_request))[0]
for word in request_json:
  print(word['lemma'])
  #TODO: compare to your schemas
  # Yifei: Should catch the key word more automatically, but here for convenience
  if word['lemma'] == 'stick':
    target_item_synset_list =  wn.synsets(word['lemma']) 
    for inventory_item, inventory_item_synset_list in inv_schema.items():
      for target_item_synset in target_item_synset_list:
        for inventory_item_synset in inventory_item_synset_list:
          similarity_score = target_item_synset.path_similarity(inventory_item_synset)
        #   print(target_item_synset, inventory_item_synset, similarity_score)  
          if similarity_score is not None:
            if similarity_score < smallest_similarity_score:
                smallest_similarity_score = similarity_score
                closest_inventory_item = inventory_item            
                closest_inventory_item_schema = [target_item_synset, inventory_item_synset]

#TODO: print the closest inventory item and what part of the schema was used to compare it
print('Clostest inventory item:', closest_inventory_item)
print('Clostest inventory item shema:', closest_inventory_item_schema[0], 'and', closest_inventory_item_schema[1])

do
you
have
any
stick
?
Clostest inventory item: oranges
Clostest inventory item shema: Synset('pin.n.05') and Synset('bergamot.n.01')
